Assuming you have already extracted the files

In [1]:
import re
#number of entries in this zip: 996181
filename= "s2-corpus-000"
filedir= "/home/ricostynha/Desktop/Erasmus/Apps/FoolingAround"
filepath = filedir+"/"+filename

In [2]:
#just for modularity read the first 10000 entries and save in a file just to have one
Maxentries = 1000000;
Maxsize = Maxentries;
dstname = "fist10000papers.txt"
dstpath = filedir+"/"+dstname;
file_list = []
dstfile= open(dstpath,"w+")
with open(filepath) as infile:
    for line in infile:
        Maxentries-=1
        file_list.append(line)
        dstfile.write(line);
        if(Maxentries==0):
            break
Maxsize = Maxsize -Maxentries
dstfile.close()

In [3]:
def parsingraw_entry(raw):
    #year is ommisee for web pages , so it will have less fields
    fields = ['"entities":','"journalVolume":','"journalPages":','"pmid":','"year":','"outCitations":','"s2Url":','"s2PdfUrl"',
         '"id":','"authors":','"journalName":','"paperAbstract":','"inCitations":','"pdfUrls":',
         '"title":','"doi":','"sources":','"doiUrl":','"venue":']
    helper_list=[]
    for field in fields:
        match = re.search(field,raw)
        if(match != None):
            helper_list.append([match.group(),match.span()])

    reference_list=[]
    for i in range(0,len(helper_list)-1):
        begin = helper_list[i][1][1]
        end   = helper_list[i+1][1][0]
        reference_list.append([helper_list[i][0],raw[begin:end]])
    begin = helper_list[-1][1][1]
    reference_list.append([helper_list[-1][0],raw[begin:]])
    return reference_list
   
    

In [5]:
#basicaly ,a lot of papers dont have the filed year
data = []
for i in range(Maxsize):
    a = parsingraw_entry(file_list[i])
    if(len(a)==19):
        c = [b[1] for b in a]
        data.append(c)
    else:
        c1 = [b[1] for b in a[0:4]]
        c2 = [None]
        c3 = [b[1] for b in a[4:]]
        data.append(c1+c2+c3)
  
        
        

In [6]:
import pandas as pd
titles  = ['entities','journalVolume','journalPages','pmid','year','outCitations','s2Url','s2PdfUrl',
'id','authors','journalName','paperAbstract','inCitations','pdfUrls',
'title','doi','sources','doiUrl','venue']

df = pd.DataFrame(data,columns=titles)

In [7]:
#replacing empty things by none
df = df.replace({"[],":None})
df = df.replace({'"",':None})
df = df.replace({':""':None})
df = df.replace({'""}\n':None})
df = df.replace({':"",':None})

In [8]:
def parse_journalVolume(string):
    if(string == None):
        return None
    # removing " " in numbers
    newstring = string[1:-2]
    #can appear in multiple journal volumes
    newstring = newstring.split(' ')
    #diferent volumes diferent array members
    return newstring
def parse_journalPages_helper(string):
    #removing \n
    if(string == None):
        return None
    string = string.replace(" ","")
    string = string.replace("\\n","")
    string = string[1:-2]
    #exist weirs formations irrefular like : S6-9;discussionS26-8  
    #(now i will parse for a range of pages)
    match = re.search('[1-9]+-[1-9]+',string)
    if(match != None):
        string = string[match.span()[0]:match.span()[1]]
        return string
    #checking if just one page
    match = re.search('[1-9]+',string)
    if(match != None):
        string = string[match.span()[0]:match.span()[1]]
        return string
    return string

def parse_journalPages(string):
    string = parse_journalPages_helper(string);
    if(string!= None):
        if(string.split('-') == []):
            print(string,string.split('-'))
        string = string.split('-')
    return string
def parse_pmid(string):
    if(string != None):
            string = string[1:-2]
    return string
def parse_year(string):
    if(string != None):
            string = string[0:-1]
    return string
def parse_outCitations(string):
    if(string ==None):
        return string
    string = string[1:-2]
    string = string.split(',')
    for i in range(len(string)):
        string[i] = string[i][1:-1]
    return string
def parse_s2Url(string):
    if(string == None):
        return string
    string = string[1:-2]
    return string
def parse_s2PdfUrl(string):
    if(string == None):
        return string
    string = string[2:-2]
    return string
def parse_id(string):
    if(string == None):
        return string
    string = string[1:-2]
    return string
    


In [9]:
def parse_authors_helper(string):
    string = string.replace('"name":','')
    string = string.replace('"ids":','')
    string = string.replace('{','')
    string = string.replace('}','')
    string = string.replace('"','')
    string = string.replace("'",'')
    
    #separate author and id
    string = string.split(',')
    return string
def parse_authors(string):
    if(string == None):
        return string
    string = string[:-1]
    string = string.replace('[','')
    string = string.replace(']','')
    string = string.split("},{")
    for i in range(len(string)):
        string[i] = parse_authors_helper(string[i])
    
def parse_journalName(string):
    if(string == None):
        return string
    return string[1:-2]
def parse_paperAbstract(string):
    if(string == None):
        return string
    #to be able to read i must replace | for another carachter
    string = string.replace('|','///')
    return string[1:-2]

def parse_incitations(string):
    return parse_outCitations(string)

def parse_pdfUrls(string):
    if(string == None):
        return string
    
    string = string[2:-3]
    string = string.replace('"','')
    string = string.replace(' ','')
    string = string.split(',')
    return string
def parse_title(string):
    if(string == None):
        return string
    return string[1:-2]
def parse_doi(string):
    if(string == None):
        return string
    return string[1:-2]
def parse_sources(string):
    if(string == None):
        return string
    string = string[1:-2]
    string = string.replace('"','')
    string = string.split(',')
    return string
def parse_doiUrl(string):
    if(string == None):
        return string
    return string[1:-2]

def parce_venue(string):
    if(string==None):
        return string
    string = string[1:-3]
    return string
    
    
    


In [10]:
df['journalVolume'] = df['journalVolume'].apply(parse_journalVolume)
df['journalPages'] = df['journalPages'].apply(parse_journalPages)
df['pmid'] = df['pmid'].apply(parse_pmid)
df['year'] = df['year'].apply(parse_year)
df['outCitations'] = df['outCitations'].apply(parse_outCitations)
df['s2Url'] = df['s2Url'].apply(parse_s2Url)
df['s2PdfUrl'] = df['s2PdfUrl'].apply(parse_s2PdfUrl)
df['id'] = df['id'].apply(parse_id)
df['authors'] = df['authors'].apply(parse_authors)
df['journalName'] = df['journalName'].apply(parse_journalName)
df['paperAbstract'] = df['paperAbstract'].apply(parse_paperAbstract)
df['inCitations'] = df['inCitations'].apply(parse_incitations)
df['pdfUrls'] = df['pdfUrls'].apply(parse_pdfUrls)
df['title'] = df['title'].apply(parse_title)
df['doi'] = df['doi'].apply(parse_doi)
df['sources'] = df['sources'].apply(parse_sources)
df['doiUrl'] = df['doiUrl'].apply(parse_doiUrl)
df['venue'] = df['venue'].apply(parce_venue)
                                
#just to being possible to wokr with stuff change None to -1
df.fillna(-1, inplace=True)


In [11]:
#creating some usefull information
def getNumcitations(x):
    if(x==-1):
        return 0
    return len(x)

df['NumOutCitations'] = df['outCitations'].apply(getNumcitations)
df['NumInCitations'] = df['inCitations'].apply(getNumcitations)

In [12]:
#No null df
Cyteddf = df[(df['NumInCitations'] != 0) & (df['NumOutCitations'] != 0) ]
del df

In [13]:
len(Cyteddf)

92770

In [14]:
Cyteddf.sort_values(by = 'NumInCitations',ascending=False)
Cyteddf = Cyteddf.reset_index()

In [16]:
inCitdic = {}
outCitdic = {}
related_ind = []
for ind in Cyteddf.index:
    for Cit in Cyteddf.loc[ind]['inCitations']:
        inCitdic[Cit] = 1 
        
    for Cit in Cyteddf.loc[ind]['outCitations']:
        outCitdic[Cit] = 1 
        
for index,idis in enumerate(Cyteddf['id']):
        if (idis in inCitdic) or (idis in outCitdic):
            related_ind.append(index)


In [19]:
related_papers = Cyteddf.loc[related_ind]

In [22]:
related_papers = related_papers.drop('index', 1)
related_papers

,entities,journalVolume,journalPages,pmid,year,outCitations,s2Url,s2PdfUrl,id,authors,...,paperAbstract,inCitations,pdfUrls,title,doi,sources,doiUrl,venue,NumOutCitations,NumInCitations
7,-1,"[148, 11]",[81],18519927,2008,"[6f2b877a5558a6b42157e6a49f068bb68787bbc5, 19f...",https://semanticscholar.org/paper/921cea2e0f57...,-1,921cea2e0f5702fc0c331e973f05aab269bb6cd7,-1,...,Context Studies in surgical patients suggest t...,"[4bbb4700f68bbd52cab186a9a27d07c3d0f81b57, f65...",-1,Sensitivity and specificity of a single emerge...,10.7326/0003-4819-148-11-200806030-00003,[Medline],https://doi.org/10.7326/0003-4819-148-11-20080...,Annals of internal medicine,51,306
12,-1,"[289, 13]",[938],24558039,2014,"[df467c552238acc95de16014b3fe10695b3775c2, 154...",https://semanticscholar.org/paper/ccc21e9a79b3...,-1,ccc21e9a79b3d94b0b7a067cebac25c92e7858aa,-1,...,Despite recent advances in understanding store...,"[ab69ac930ee10ebd55280221c3f8824f8a4f8d0e, a0f...",-1,Reticulon 4 is necessary for endoplasmic retic...,10.1074/jbc.M114.548602,[Medline],https://doi.org/10.1074/jbc.M114.548602,The Journal of biological chemistry,65,16
18,-1,-1,"[33, 4]",-1,2013,"[e69b6f374b073df9f37c4bffe5d5056ed7fdbab4, 618...",https://semanticscholar.org/paper/7a932e847292...,-1,7a932e84729272c4b0009ecd93936f61d95ca0cb,-1,...,The ability to use free-hand gestures is extre...,"[15f7eeb586d58ecfb3e1398b53d97981d456456a, 321...",[https://www.wikidata.org/entity/Q57691797],Free-hands interaction in augmented reality,10.1145/2491367.2491370,[DBLP],https://doi.org/10.1145/2491367.2491370,SUI,38,19
25,-1,[27],[5],-1,2010,"[2c3805411040bee0856c0f2893d94f768c948a89, be7...",https://semanticscholar.org/paper/837eacb96871...,-1,837eacb9687137ea326d1dc5cd969f4b212ce51f,-1,...,This article shows the potential for convex op...,"[c1871e57f75f488faacd5d79abe4bc1bf12faeac, 01c...","[https://doi.org/10.1109/MSP.2010.936020, http...",Real-Time Convex Optimization in Signal Proces...,10.1109/MSP.2010.936020,[DBLP],https://doi.org/10.1109/MSP.2010.936020,IEEE Signal Processing Magazine,68,179
40,-1,-1,-1,-1,2018,"[8bddfb749800ba2e40ea460164d2dc1df4e3f33b, ce5...",https://semanticscholar.org/paper/59a3a6b3ceed...,http://pdfs.semanticscholar.org/59a3/a6b3ceed5...,59a3a6b3ceed5430fb8b0040f0ae946bf30d1b00,-1,...,2 Department of Biochemistry and Structural Bi...,[97ab64a7a5f4c4b47c56e5ac5b320a2df8214121],[https://www.preprints.org/manuscript/201804.0...,) 1 Nanomedicine and Phage Capsids 2,-1,-1,-1,-1,53,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92738,-1,-1,-1,-1,1991,"[f24a7d17cdaef589bac433386a299b20be7955ed, 8c5...",https://semanticscholar.org/paper/9843a1adedb7...,http://pdfs.semanticscholar.org/93c3/e5c95569c...,9843a1adedb7e2b0aec3ea540c5fa625f4d8e7db,-1,...,"For a nonnegative integer k, let s(k) denote t...","[704c970eff73e62bedec4cdbb14ff08ccb8eb4ad, 93d...",[http://www.mathstat.dal.ca/FQ/Scanned/29-2/ke...,An extension of a theorem by Cheo and Yien con...,-1,-1,-1,-1,2,10
92744,-1,"[32, Database, issue]",[3],14681418,2004,"[81e8f8d7f39b0e0b1de10ac98fe91f3c1efa4360, 43e...",https://semanticscholar.org/paper/20b1e53a08fb...,http://pdfs.semanticscholar.org/b08b/bbb71af7a...,20b1e53a08fbff57ef1d0b58ae614f5c0b9799da,-1,...,Using more than one approach to characterizing...,"[89ae9ed4dfe5f34a8a8e260045048e2895f3fe4f, 093...",[https://www.researchgate.net/profile/Margreth...,GenProtEC: an updated and improved analysis of...,10.1093/nar/gkh087,"[Medline, DBLP]",https://doi.org/10.1093/nar%2Fgkh087,Nucleic Acids Research,10,72
92745,-1,[17],[6],-1,2005,"[e8a8d7585e71e637e78bb5d4915def4f067d4537, a72...",https://semanticscholar.org/paper/380ea98e2395...,http://pdfs.semanticscholar.org/380e/a98e2395b...,380ea98e2395b5f48dc54a683ccfa9ce02c3dbdc,-1,...,"Rose, Jones and Truex (2005) offer a valuable ...","[e62aeee59c5ffe382e0b9baeb6a3305db3fe3314, 402...",[http://aisel.aisnet.org/cgi/viewcontent.cgi?a...,Material Works: Exploring the Situated Entangl

In [23]:
related_papers.to_csv("/home/ricostynha/Desktop/first1000entries",sep = "|",na_rep = "* *",index = False)
